In [1]:
import numpy as np
import pandas as pd
import os
import glob
import superskrypt as sp
from scipy import stats

In [2]:
PTH = r"C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_EEG"
os.chdir(PTH)
print(os.getcwd())

C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE_EEG


In [3]:
#for f in glob.glob('*.txt'):
#     print(f)

In [4]:
def is_odd(num):
    return num % 2

In [96]:
d2 = pd.DataFrame()
d_all = pd.DataFrame()
subj_num = []
for f in glob.glob('*.txt'):
    subj_num.append(int(f.split('.')[0][1:]))
    df = pd.read_csv(f, sep='\t')
    print('read {}!'.format(f))
    db = sp.get_color_info(df)
    d2 = d2.append(db, ignore_index=True)
    d_all = d_all.append(db, ignore_index=True)
print('DONE!')

read s1.txt!
read s2.txt!
DONE!


In [97]:
for c in d2.columns:
    for i in d2.index:
        d2.loc[i, c] = np.mean(d2.loc[i, c])

In [98]:
for c in d2.columns:
    for i in d2.index:
        d2.loc[i, 'comp'] = (d2.loc[i, 'comp_l']+d2.loc[i, 'comp_r'])/2
        d2.loc[i, 'incomp'] = (d2.loc[i, 'incomp_l']+d2.loc[i, 'incomp_r'])/2
        d2.loc[i, 'neut'] = (d2.loc[i, 'neut_l']+d2.loc[i, 'neut_r'])/2

In [99]:
d2.index = subj_num
for i in d2.index:
    if is_odd(i):
        d2.loc[i, 'condition'] = 'info'
    else:
        d2.loc[i, 'condition'] = 'noinfo'

In [100]:
# Derives all SoA for each codition (comp/neut/incomp), 
# each hand (l/r) and each block (1-6) separately
# and makes a separate column for them

for col in d_all.columns:
    for i in range(0,6):
        d2.loc[:, '{}_{}'.format(col, i+1)] = d_all.loc[0, col][i]

In [101]:
d2

,comp_l,comp_r,incomp_l,incomp_r,neut_l,neut_r,comp,incomp,neut,condition,...,neut_l_3,neut_l_4,neut_l_5,neut_l_6,neut_r_1,neut_r_2,neut_r_3,neut_r_4,neut_r_5,neut_r_6
1,59.83333,55.83333,50,56.66667,49.16667,67.5,57.833333,53.333333,58.333333,info,...,100,80,10,40,85,40,40,40,100,100
2,59.83333,55.83333,50,56.66667,49.16667,67.5,57.833333,53.333333,58.333333,noinfo,...,100,80,10,40,85,40,40,40,100,100


In [94]:
d2 = d2.loc[np.sort(d2.index), :]
d2.to_excel('SoA_EEG.xlsx')